In [1]:
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

Using TensorFlow backend.


# Parameters Config

In [2]:
class colors:
    ok='\033[92m'
    fail='\033[91m'
    close='\033[0m'

In [3]:
TRAINING_SIZE=80000
DIGITS=4
REVERSE=False
MAXLEN=DIGITS+1+DIGITS
chars='0123456789+ '
RNN=layers.LSTM
HIDDEN_SIZE=128
BATCH_SIZE=128
LAYERS=1

In [4]:
class CharacterTable(object):
    def __init__(self,chars):
        self.chars=sorted(set(chars))
        self.char_indices=dict((c,i) for i,c in enumerate(self.chars))
        self.indices_char=dict((i,c) for i,c in enumerate(self.chars))
    
    def encode(self,C,num_rows): #input C is string
        x=np.zeros((num_rows, len(self.chars)))
        for i,c in enumerate(C):
            x[i,self.char_indices[c]]=1
        return x
    
    def decode(self,x,calc_argmax=True): #if input x is 1-D array then culc_argmax=True； otherwise, if x is string or int then False
        if calc_argmax:
            x=x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)
    

In [5]:
ctable=CharacterTable(chars)

In [6]:
print(ctable.indices_char)
print(ctable.char_indices)

{0: ' ', 1: '+', 2: '0', 3: '1', 4: '2', 5: '3', 6: '4', 7: '5', 8: '6', 9: '7', 10: '8', 11: '9'}
{' ': 0, '+': 1, '0': 2, '1': 3, '2': 4, '3': 5, '4': 6, '5': 7, '6': 8, '7': 9, '8': 10, '9': 11}


# Data Generation

In [7]:
questions=[]
expected=[]
seen=set()
print('Generating data....')
while len(questions)<TRAINING_SIZE:
    f=lambda:int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1,DIGITS+1))))
    a,b=f(),f()
    key=tuple(sorted((a,b)))
    if key in seen:
        continue
    seen.add(key)
    q='{}+{}'.format(a,b)
    query=q+' '*(MAXLEN-len(q))
    ans=str(a+b)
    ans+=' '*(DIGITS+1-len(ans))
    if REVERSE:
        query=query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:',len(questions))

Generating data....
Total addition questions: 80000


In [8]:
print(questions[:5],expected[:5])

['382+6    ', '57+43    ', '72+51    ', '1294+983 ', '4+9      '] ['388  ', '100  ', '123  ', '2277 ', '13   ']


# Processing

In [9]:
print('Vectorization....')
x=np.zeros((len(questions),MAXLEN,len(chars)),dtype=np.bool)
y=np.zeros((len(expected),DIGITS+1,len(chars)),dtype=np.bool)
for i,sentence in enumerate(questions):
    x[i]=ctable.encode(sentence,MAXLEN)
for i,sentence in enumerate(expected):
    y[i]=ctable.encode(sentence,DIGITS+1)

Vectorization....


In [10]:
indices=np.arange(len(y))
np.random.shuffle(indices)
x=x[indices]
y=y[indices]

#train_test_split
train_x=x[:20000]
train_y=y[:20000]
test_x=x[20000:]
test_y=y[20000:]

split_at=len(train_x)-len(train_x)//10
(x_train,x_val)=train_x[:split_at],train_x[split_at:]
(y_train,y_val)=train_y[:split_at],train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Date:')
print(test_x.shape)
print(test_y.shape)

Training Data:
(18000, 9, 12)
(18000, 5, 12)
Validation Data:
(2000, 9, 12)
(2000, 5, 12)
Testing Date:
(60000, 9, 12)
(60000, 5, 12)


# Build Model

In [11]:
print('Build model...')

model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))
for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))
model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 5, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 5, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 5, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


# Training and Testing

In [12]:
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 0
Instructions for updating:
Use tf.cast instead.
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 6s 334us/step - loss: 1.9351 - acc: 0.3172 - val_loss: 1.7908 - val_acc: 0.3518
Q 629+4269  T 4898  ☒ 1100 
Q 866+1042  T 1908  ☒ 1100 
Q 2417+43   T 2460  ☒ 110  
Q 0+4223    T 4223  ☒ 115  
Q 99+5      T 104   ☒ 115  
Q 270+32    T 302   ☒ 115  
Q 5+157     T 162   ☒ 115  
Q 4953+173  T 5126  ☒ 1100 
Q 2907+6252 T 9159  ☒ 1100 
Q 6974+331  T 7305  ☒ 1100 

--------------------------------------------------
Iteration 1
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 185us/step - loss: 1.7850 - acc: 0.3497 - val_loss: 1.7776 - val_acc: 0.3470
Q 651+166   T 817   ☒ 1451 
Q 99+918    T 1017  ☒ 1999 
Q 646+1679  T 2325  ☒ 1404 
Q 8803+93   T 8896  ☒ 1409 
Q 8619+496  T 9115  ☒ 1409 
Q 11+8031   T 8042  ☒ 1211 
Q 59

18000/18000 [==============================] - 3s 193us/step - loss: 1.2344 - acc: 0.5383 - val_loss: 1.2331 - val_acc: 0.5346
Q 4105+576  T 4681  ☒ 4592 
Q 63+17     T 80    ☒ 70   
Q 62+7321   T 7383  ☒ 7304 
Q 665+3139  T 3804  ☒ 3004 
Q 7119+1726 T 8845  ☒ 7104 
Q 86+455    T 541   ☒ 524  
Q 9+537     T 546   ☒ 573  
Q 144+3     T 147   ☒ 144  
Q 614+49    T 663   ☒ 644  
Q 301+3     T 304   ☒ 311  

--------------------------------------------------
Iteration 14
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 206us/step - loss: 1.1969 - acc: 0.5525 - val_loss: 1.2023 - val_acc: 0.5481
Q 15+8421   T 8436  ☒ 8521 
Q 69+831    T 900   ☒ 822  
Q 345+3     T 348   ☒ 349  
Q 7667+5    T 7672  ☒ 7671 
Q 3444+2125 T 5569  ☒ 5611 
Q 440+50    T 490   ☒ 565  
Q 2588+3    T 2591  ☒ 2595 
Q 515+687   T 1202  ☒ 1313 
Q 9+2754    T 2763  ☒ 2733 
Q 0+422     T 422   ☑ 422  

--------------------------------------------------
Iteration 

Q 4+5502    T 5506  ☒ 5518 
Q 17+5837   T 5854  ☒ 5847 
Q 514+6955  T 7469  ☒ 7115 
Q 574+129   T 703   ☒ 783  
Q 912+2904  T 3816  ☒ 3139 
Q 6518+39   T 6557  ☒ 6581 
Q 2261+935  T 3196  ☒ 3164 
Q 75+1734   T 1809  ☒ 1822 
Q 646+1679  T 2325  ☒ 2333 
Q 21+91     T 112   ☒ 110  

--------------------------------------------------
Iteration 27
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 200us/step - loss: 0.9094 - acc: 0.6663 - val_loss: 0.9826 - val_acc: 0.6276
Q 777+738   T 1515  ☑ 1515 
Q 5439+6240 T 11679 ☒ 12836
Q 9213+9    T 9222  ☒ 9234 
Q 9306+6    T 9312  ☒ 9309 
Q 1708+7919 T 9627  ☒ 1644 
Q 9+212     T 221   ☑ 221  
Q 227+755   T 982   ☒ 801  
Q 8091+7    T 8098  ☒ 8089 
Q 450+6028  T 6478  ☒ 6662 
Q 1236+628  T 1864  ☒ 1843 

--------------------------------------------------
Iteration 28
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 191us/step - lo

18000/18000 [==============================] - 3s 173us/step - loss: 0.6948 - acc: 0.7462 - val_loss: 0.8500 - val_acc: 0.6753
Q 77+9358   T 9435  ☒ 9464 
Q 19+50     T 69    ☒ 77   
Q 2967+509  T 3476  ☒ 3407 
Q 57+215    T 272   ☑ 272  
Q 4753+1    T 4754  ☑ 4754 
Q 76+539    T 615   ☒ 637  
Q 8510+8959 T 17469 ☒ 16503
Q 97+27     T 124   ☒ 125  
Q 576+29    T 605   ☒ 507  
Q 39+6513   T 6552  ☒ 6562 

--------------------------------------------------
Iteration 41
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 195us/step - loss: 0.6777 - acc: 0.7539 - val_loss: 0.8378 - val_acc: 0.6773
Q 9862+7993 T 17855 ☒ 18701
Q 34+5297   T 5331  ☒ 5311 
Q 762+709   T 1471  ☒ 1428 
Q 684+3542  T 4226  ☒ 4209 
Q 453+74    T 527   ☒ 509  
Q 281+99    T 380   ☒ 378  
Q 342+19    T 361   ☑ 361  
Q 96+2772   T 2868  ☒ 2848 
Q 73+61     T 134   ☒ 133  
Q 15+674    T 689   ☒ 788  

--------------------------------------------------
Iteration 

Q 655+66    T 721   ☑ 721  
Q 6974+331  T 7305  ☒ 7196 
Q 113+2895  T 3008  ☒ 3000 
Q 1+3274    T 3275  ☒ 3274 
Q 3779+4141 T 7920  ☒ 7209 
Q 4399+0    T 4399  ☒ 4390 
Q 144+3     T 147   ☑ 147  
Q 837+9156  T 9993  ☒ 9033 
Q 59+4300   T 4359  ☒ 4398 
Q 4623+66   T 4689  ☑ 4689 

--------------------------------------------------
Iteration 54
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 210us/step - loss: 0.4701 - acc: 0.8338 - val_loss: 0.7027 - val_acc: 0.7299
Q 256+29    T 285   ☑ 285  
Q 0+837     T 837   ☑ 837  
Q 8529+8    T 8537  ☑ 8537 
Q 23+420    T 443   ☒ 442  
Q 2344+43   T 2387  ☒ 2487 
Q 3+5370    T 5373  ☑ 5373 
Q 8257+3    T 8260  ☑ 8260 
Q 18+666    T 684   ☑ 684  
Q 21+680    T 701   ☑ 701  
Q 29+9637   T 9666  ☒ 9646 

--------------------------------------------------
Iteration 55
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 211us/step - lo

Q 240+8726  T 8966  ☒ 8886 
Q 920+4594  T 5514  ☒ 5494 
Q 3+565     T 568   ☑ 568  
Q 19+59     T 78    ☑ 78   

--------------------------------------------------
Iteration 67
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 190us/step - loss: 0.3154 - acc: 0.8965 - val_loss: 0.6421 - val_acc: 0.7605
Q 325+2741  T 3066  ☑ 3066 
Q 0+502     T 502   ☑ 502  
Q 610+4278  T 4888  ☒ 4989 
Q 199+230   T 429   ☒ 410  
Q 4365+8    T 4373  ☒ 4372 
Q 23+8114   T 8137  ☒ 8186 
Q 4940+5    T 4945  ☑ 4945 
Q 2074+1105 T 3179  ☒ 3386 
Q 2075+5887 T 7962  ☒ 8862 
Q 9371+55   T 9426  ☒ 9386 

--------------------------------------------------
Iteration 68
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 196us/step - loss: 0.3055 - acc: 0.9016 - val_loss: 0.6253 - val_acc: 0.7736
Q 5+4276    T 4281  ☑ 4281 
Q 9371+55   T 9426  ☒ 9386 
Q 646+1679  T 2325  ☒ 2344 
Q 21+862    T 883   ☒ 

18000/18000 [==============================] - 3s 187us/step - loss: 0.2070 - acc: 0.9387 - val_loss: 0.6425 - val_acc: 0.7822
Q 8140+5    T 8145  ☒ 8144 
Q 73+826    T 899   ☑ 899  
Q 63+229    T 292   ☒ 282  
Q 29+74     T 103   ☑ 103  
Q 1786+1977 T 3763  ☒ 2674 
Q 5884+43   T 5927  ☒ 5917 
Q 5+4273    T 4278  ☒ 4288 
Q 36+118    T 154   ☒ 164  
Q 9+9368    T 9377  ☑ 9377 
Q 639+9     T 648   ☑ 648  

--------------------------------------------------
Iteration 81
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 186us/step - loss: 0.1946 - acc: 0.9448 - val_loss: 0.6449 - val_acc: 0.7814
Q 86+184    T 270   ☒ 269  
Q 0+502     T 502   ☑ 502  
Q 453+74    T 527   ☑ 527  
Q 1423+2003 T 3426  ☒ 3655 
Q 5801+4400 T 10201 ☒ 9010 
Q 1+477     T 478   ☑ 478  
Q 9+3472    T 3481  ☒ 3480 
Q 9063+861  T 9924  ☒ 9815 
Q 363+18    T 381   ☑ 381  
Q 4910+8    T 4918  ☒ 4908 

--------------------------------------------------
Iteration 

Q 18+86     T 104   ☑ 104  
Q 9513+8577 T 18090 ☒ 18210
Q 1+1941    T 1942  ☑ 1942 
Q 813+5810  T 6623  ☒ 6644 
Q 49+219    T 268   ☑ 268  
Q 436+5     T 441   ☑ 441  
Q 4586+6    T 4592  ☑ 4592 
Q 49+8018   T 8067  ☒ 8077 
Q 32+4579   T 4611  ☑ 4611 
Q 875+6     T 881   ☒ 891  

--------------------------------------------------
Iteration 94
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 186us/step - loss: 0.1197 - acc: 0.9728 - val_loss: 0.7130 - val_acc: 0.7891
Q 5487+338  T 5825  ☒ 5945 
Q 963+7257  T 8220  ☒ 8130 
Q 689+856   T 1545  ☒ 1585 
Q 86+35     T 121   ☒ 122  
Q 298+9772  T 10070 ☒ 10110
Q 7673+7    T 7680  ☑ 7680 
Q 5626+444  T 6070  ☒ 6090 
Q 2925+59   T 2984  ☒ 2905 
Q 4753+1    T 4754  ☑ 4754 
Q 1+6669    T 6670  ☒ 6660 

--------------------------------------------------
Iteration 95
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 194us/step - lo

In [13]:
print("MSG : Prediction")

pred_y=model.predict_classes(test_x,verbose=0)
right=0
for i in range(len(test_x)):
    q=ctable.decode(test_x[i])
    correct=ctable.decode(test_y[i])
    pred=ctable.decode(pred_y[i], calc_argmax=False)
    print('Q', q[::-1] if REVERSE else q, end=' ')
    print('T', correct, end=' ')
    if correct==pred:
        right+=1
        print(colors.ok+'☑' + colors.close, end=' ')
    else:
        print(colors.fail + '☒' + colors.close, end=' ')
    print(pred)
print('accuracy=',(right/len(test_x))*100,'%')

MSG : Prediction
Q 1263+9    T 1272  ☑ 1272 
Q 2+9914    T 9916  ☒ 9917 
Q 8525+54   T 8579  ☒ 8589 
Q 9292+4    T 9296  ☒ 9236 
Q 186+3230  T 3416  ☒ 3525 
Q 3291+2    T 3293  ☑ 3293 
Q 131+8935  T 9066  ☒ 8976 
Q 6789+5    T 6794  ☒ 6784 
Q 18+7052   T 7070  ☒ 7181 
Q 83+46     T 129   ☑ 129  
Q 281+5547  T 5828  ☒ 5898 
Q 919+5048  T 5967  ☒ 6046 
Q 5595+110  T 5705  ☒ 5603 
Q 71+3008   T 3079  ☑ 3079 
Q 288+49    T 337   ☑ 337  
Q 2+132     T 134   ☒ 135  
Q 485+4933  T 5418  ☒ 5417 
Q 6922+258  T 7180  ☒ 7100 
Q 552+528   T 1080  ☒ 1090 
Q 91+4765   T 4856  ☑ 4856 
Q 4+7570    T 7574  ☑ 7574 
Q 93+773    T 866   ☒ 865  
Q 11+323    T 334   ☑ 334  
Q 33+19     T 52    ☒ 51   
Q 9845+117  T 9962  ☒ 90622
Q 932+4181  T 5113  ☒ 4183 
Q 8292+95   T 8387  ☒ 8907 
Q 204+90    T 294   ☒ 295  
Q 89+648    T 737   ☒ 747  
Q 15+704    T 719   ☑ 719  
Q 1813+3    T 1816  ☑ 1816 
Q 648+88    T 736   ☒ 746  
Q 4987+573  T 5560  ☒ 5569 
Q 897+461   T 1358  ☒ 1458 
Q 55+945    T 1000  ☑ 1000 
Q 6

Q 73+97     T 170   ☑ 170  
Q 3299+1531 T 4830  ☒ 5610 
Q 4415+378  T 4793  ☒ 4823 
Q 840+5840  T 6680  ☒ 9480 
Q 7217+6390 T 13607 ☒ 13216
Q 5111+4    T 5115  ☒ 5114 
Q 2213+2680 T 4893  ☒ 4904 
Q 280+44    T 324   ☑ 324  
Q 819+423   T 1242  ☑ 1242 
Q 594+7797  T 8391  ☒ 8580 
Q 913+92    T 1005  ☒ 9955 
Q 2545+6336 T 8881  ☒ 8870 
Q 61+36     T 97    ☒ 98   
Q 2378+5345 T 7723  ☒ 8942 
Q 3+526     T 529   ☒ 539  
Q 79+6549   T 6628  ☒ 6638 
Q 385+8     T 393   ☑ 393  
Q 8538+3480 T 12018 ☒ 12058
Q 932+405   T 1337  ☒ 1347 
Q 332+872   T 1204  ☒ 1193 
Q 9+196     T 205   ☒ 206  
Q 421+36    T 457   ☒ 447  
Q 471+329   T 800   ☒ 700  
Q 1126+513  T 1639  ☒ 1779 
Q 902+1     T 903   ☑ 903  
Q 122+580   T 702   ☑ 702  
Q 2365+7    T 2372  ☑ 2372 
Q 998+88    T 1086  ☒ 1076 
Q 1973+66   T 2039  ☒ 2098 
Q 68+2897   T 2965  ☑ 2965 
Q 16+577    T 593   ☑ 593  
Q 2407+75   T 2482  ☒ 2423 
Q 4+573     T 577   ☒ 578  
Q 7346+88   T 7434  ☑ 7434 
Q 622+3627  T 4249  ☒ 5998 
Q 480+27    T 507   

Q 5328+5    T 5333  ☑ 5333 
Q 2+3314    T 3316  ☑ 3316 
Q 77+224    T 301   ☑ 301  
Q 80+24     T 104   ☑ 104  
Q 63+9290   T 9353  ☒ 9333 
Q 0+2736    T 2736  ☑ 2736 
Q 16+6514   T 6530  ☒ 6559 
Q 59+7946   T 8005  ☒ 8015 
Q 2127+6771 T 8898  ☒ 8997 
Q 17+8727   T 8744  ☑ 8744 
Q 110+7906  T 8016  ☒ 7965 
Q 4+5302    T 5306  ☑ 5306 
Q 1+5335    T 5336  ☑ 5336 
Q 7594+63   T 7657  ☒ 7946 
Q 8289+541  T 8830  ☒ 8701 
Q 654+59    T 713   ☒ 703  
Q 50+37     T 87    ☒ 77   
Q 76+4544   T 4620  ☒ 4540 
Q 2+258     T 260   ☑ 260  
Q 8+9299    T 9307  ☒ 9297 
Q 370+0     T 370   ☑ 370  
Q 9+534     T 543   ☑ 543  
Q 26+107    T 133   ☒ 123  
Q 439+64    T 503   ☑ 503  
Q 44+634    T 678   ☒ 677  
Q 31+4429   T 4460  ☒ 4360 
Q 8+6932    T 6940  ☒ 6949 
Q 837+2060  T 2897  ☒ 2877 
Q 373+6     T 379   ☑ 379  
Q 66+986    T 1052  ☒ 1043 
Q 3+697     T 700   ☑ 700  
Q 661+33    T 694   ☑ 694  
Q 24+94     T 118   ☑ 118  
Q 6117+8    T 6125  ☑ 6125 
Q 940+1457  T 2397  ☒ 9069 
Q 571+350   T 921   

Q 742+357   T 1099  ☒ 1009 
Q 2920+92   T 3012  ☒ 3911 
Q 744+2601  T 3345  ☒ 3464 
Q 9644+1    T 9645  ☑ 9645 
Q 63+5554   T 5617  ☒ 5607 
Q 6228+2    T 6230  ☒ 6220 
Q 2331+7072 T 9403  ☒ 9024 
Q 3272+6    T 3278  ☑ 3278 
Q 100+93    T 193   ☑ 193  
Q 9424+71   T 9495  ☑ 9495 
Q 7385+55   T 7440  ☒ 7459 
Q 1249+808  T 2057  ☒ 2177 
Q 868+39    T 907   ☒ 927  
Q 7601+9500 T 17101 ☒ 16000
Q 11+963    T 974   ☑ 974  
Q 2680+4    T 2684  ☒ 2673 
Q 320+9     T 329   ☑ 329  
Q 236+482   T 718   ☒ 708  
Q 6477+24   T 6501  ☑ 6501 
Q 43+358    T 401   ☒ 400  
Q 6181+107  T 6288  ☑ 6288 
Q 51+6158   T 6209  ☒ 6218 
Q 65+252    T 317   ☒ 307  
Q 569+9681  T 10250 ☒ 10329
Q 563+7929  T 8492  ☒ 8453 
Q 803+4444  T 5247  ☑ 5247 
Q 752+3013  T 3765  ☒ 3764 
Q 511+7204  T 7715  ☒ 7634 
Q 917+84    T 1001  ☒ 990  
Q 75+9784   T 9859  ☒ 9849 
Q 7679+157  T 7836  ☒ 7956 
Q 38+56     T 94    ☑ 94   
Q 15+924    T 939   ☒ 929  
Q 905+95    T 1000  ☒ 999  
Q 810+454   T 1264  ☒ 1154 
Q 8+749     T 757   

Q 1953+745  T 2698  ☒ 2608 
Q 283+61    T 344   ☑ 344  
Q 731+83    T 814   ☒ 804  
Q 31+463    T 494   ☒ 484  
Q 31+55     T 86    ☑ 86   
Q 783+1     T 784   ☑ 784  
Q 5726+7555 T 13281 ☒ 13941
Q 65+669    T 734   ☒ 744  
Q 667+4     T 671   ☑ 671  
Q 976+639   T 1615  ☒ 1635 
Q 852+119   T 971   ☑ 971  
Q 8+7849    T 7857  ☒ 7847 
Q 8+4209    T 4217  ☒ 4226 
Q 1+824     T 825   ☑ 825  
Q 2668+0    T 2668  ☒ 2667 
Q 77+385    T 462   ☑ 462  
Q 443+0     T 443   ☑ 443  
Q 7+3250    T 3257  ☑ 3257 
Q 1+4083    T 4084  ☑ 4084 
Q 485+756   T 1241  ☒ 1261 
Q 808+3206  T 4014  ☒ 4925 
Q 300+9063  T 9363  ☒ 9534 
Q 860+3383  T 4243  ☒ 4564 
Q 863+81    T 944   ☒ 954  
Q 191+7     T 198   ☒ 197  
Q 108+27    T 135   ☑ 135  
Q 180+95    T 275   ☒ 265  
Q 5587+808  T 6395  ☒ 6315 
Q 3745+79   T 3824  ☒ 3834 
Q 9900+22   T 9922  ☑ 9922 
Q 602+216   T 818   ☒ 827  
Q 163+8994  T 9157  ☒ 9059 
Q 2173+14   T 2187  ☒ 2197 
Q 753+570   T 1323  ☒ 1333 
Q 9105+803  T 9908  ☒ 9818 
Q 6183+1    T 6184  

...

Q 6781+921  T 7702  ☒ 7622 
Q 0+394     T 394   ☑ 394  
Q 1394+6540 T 7934  ☒ 8844 
Q 3259+9957 T 13216 ☒ 13225
Q 4470+560  T 5030  ☒ 4150 
Q 146+33    T 179   ☑ 179  
Q 1+9843    T 9844  ☑ 9844 
Q 744+15    T 759   ☒ 768  
Q 8609+2    T 8611  ☑ 8611 
Q 64+64     T 128   ☑ 128  
Q 88+113    T 201   ☑ 201  
Q 1117+31   T 1148  ☒ 1139 
Q 33+17     T 50    ☒ 40   
Q 921+3149  T 4070  ☒ 4149 
Q 35+73     T 108   ☑ 108  
Q 4863+3    T 4866  ☒ 4857 
Q 4+4380    T 4384  ☒ 4484 
Q 8478+915  T 9393  ☒ 9564 
Q 215+8683  T 8898  ☒ 8997 
Q 3307+691  T 3998  ☒ 3988 
Q 48+65     T 113   ☒ 123  
Q 249+5937  T 6186  ☒ 6164 
Q 481+12    T 493   ☒ 498  
Q 830+5     T 835   ☒ 834  
Q 1285+9    T 1294  ☒ 1204 
Q 22+143    T 165   ☑ 165  
Q 323+81    T 404   ☑ 404  
Q 0+2177    T 2177  ☑ 2177 
Q 6+3899    T 3905  ☑ 3905 
Q 943+2934  T 3877  ☒ 9807 
Q 51+39     T 90    ☒ 89   
Q 666+2518  T 3184  ☒ 3104 
Q 33+6885   T 6918  ☒ 7908 
Q 13+24     T 37    ☑ 37   
Q 35+1201   T 1236  ☒ 1246 
Q 95+39     T 134   

Q 3+494     T 497   ☑ 497  
Q 4+5112    T 5116  ☑ 5116 
Q 305+21    T 326   ☑ 326  
Q 886+88    T 974   ☑ 974  
Q 93+9033   T 9126  ☒ 9196 
Q 57+848    T 905   ☒ 915  
Q 8+183     T 191   ☑ 191  
Q 35+746    T 781   ☑ 781  
Q 293+7022  T 7315  ☒ 7325 
Q 8549+434  T 8983  ☒ 9903 
Q 2315+965  T 3280  ☒ 3199 
Q 447+5605  T 6052  ☑ 6052 
Q 7+7890    T 7897  ☑ 7897 
Q 518+9558  T 10076 ☒ 10175
Q 786+6818  T 7604  ☒ 7504 
Q 550+231   T 781   ☒ 882  
Q 738+9246  T 9984  ☒ 9935 
Q 57+314    T 371   ☑ 371  
Q 1348+1811 T 3159  ☒ 3308 
Q 445+35    T 480   ☑ 480  
Q 10+594    T 604   ☒ 613  
Q 72+9688   T 9760  ☒ 9940 
Q 123+9774  T 9897  ☒ 9804 
Q 6+9205    T 9211  ☒ 9202 
Q 56+233    T 289   ☑ 289  
Q 9+8070    T 8079  ☑ 8079 
Q 961+108   T 1069  ☒ 1089 
Q 687+66    T 753   ☑ 753  
Q 7476+7399 T 14875 ☒ 15245
Q 7063+6594 T 13657 ☒ 13268
Q 85+6105   T 6190  ☒ 6119 
Q 50+73     T 123   ☑ 123  
Q 2115+1    T 2116  ☒ 2117 
Q 8+8057    T 8065  ☒ 8066 
Q 333+7235  T 7568  ☒ 7668 
Q 628+48    T 676   

Q 5+2972    T 2977  ☑ 2977 
Q 176+4449  T 4625  ☒ 4535 
Q 940+5     T 945   ☑ 945  
Q 25+5390   T 5415  ☒ 5315 
Q 5200+2531 T 7731  ☒ 8532 
Q 8418+714  T 9132  ☒ 9153 
Q 9319+2    T 9321  ☒ 9312 
Q 11+4034   T 4045  ☒ 4035 
Q 99+1324   T 1423  ☑ 1423 
Q 4249+254  T 4503  ☒ 4474 
Q 4+4966    T 4970  ☒ 4979 
Q 38+7114   T 7152  ☒ 7172 
Q 18+4607   T 4625  ☒ 4636 
Q 4684+5851 T 10535 ☒ 10346
Q 970+925   T 1895  ☒ 1914 
Q 207+125   T 332   ☒ 242  
Q 85+1910   T 1995  ☒ 1985 
Q 1191+6    T 1197  ☒ 1196 
Q 6194+0    T 6194  ☑ 6194 
Q 9+4174    T 4183  ☒ 4184 
Q 5816+2    T 5818  ☑ 5818 
Q 51+24     T 75    ☒ 74   
Q 67+5957   T 6024  ☑ 6024 
Q 13+238    T 251   ☑ 251  
Q 47+8094   T 8141  ☒ 8131 
Q 2+3253    T 3255  ☑ 3255 
Q 3+8876    T 8879  ☑ 8879 
Q 78+4965   T 5043  ☒ 5053 
Q 942+154   T 1096  ☒ 1066 
Q 3141+3    T 3144  ☒ 3134 
Q 130+289   T 419   ☒ 499  
Q 493+80    T 573   ☒ 583  
Q 475+7     T 482   ☑ 482  
Q 6276+52   T 6328  ☒ 6318 
Q 8516+66   T 8582  ☒ 8592 
Q 4361+8014 T 12375 

Q 7179+3    T 7182  ☑ 7182 
Q 5+3990    T 3995  ☑ 3995 
Q 81+997    T 1078  ☒ 1077 
Q 2+351     T 353   ☑ 353  
Q 9862+5    T 9867  ☒ 9877 
Q 9+98      T 107   ☒ 187  
Q 2421+1    T 2422  ☑ 2422 
Q 22+8435   T 8457  ☒ 8437 
Q 35+6439   T 6474  ☒ 6443 
Q 333+137   T 470   ☑ 470  
Q 68+3974   T 4042  ☒ 4052 
Q 28+251    T 279   ☒ 289  
Q 3+5534    T 5537  ☑ 5537 
Q 5+3149    T 3154  ☒ 3144 
Q 60+263    T 323   ☑ 323  
Q 55+74     T 129   ☑ 129  
Q 7527+45   T 7572  ☒ 7582 
Q 1+261     T 262   ☑ 262  
Q 130+57    T 187   ☑ 187  
Q 0+6690    T 6690  ☑ 6690 
Q 915+426   T 1341  ☒ 1351 
Q 7+9475    T 9482  ☒ 9472 
Q 795+95    T 890   ☑ 890  
Q 5703+75   T 5778  ☒ 5788 
Q 1380+3    T 1383  ☑ 1383 
Q 5443+63   T 5506  ☒ 5576 
Q 8646+657  T 9303  ☒ 9233 
Q 9689+3843 T 13532 ☒ 13772
Q 0+535     T 535   ☑ 535  
Q 81+3      T 84    ☑ 84   
Q 820+211   T 1031  ☒ 1022 
Q 991+78    T 1069  ☑ 1069 
Q 8+4660    T 4668  ☑ 4668 
Q 7901+5585 T 13486 ☒ 14984
Q 6891+534  T 7425  ☒ 7344 
Q 473+303   T 776   

Q 14+56     T 70    ☑ 70   
Q 7144+291  T 7435  ☒ 7445 
Q 347+68    T 415   ☑ 415  
Q 2986+0    T 2986  ☒ 2976 
Q 185+0     T 185   ☑ 185  
Q 805+152   T 957   ☒ 947  
Q 78+7988   T 8066  ☒ 8056 
Q 87+3552   T 3639  ☑ 3639 
Q 600+362   T 962   ☒ 992  
Q 5+4724    T 4729  ☑ 4729 
Q 61+4356   T 4417  ☒ 4427 
Q 360+8     T 368   ☑ 368  
Q 532+643   T 1175  ☑ 1175 
Q 4078+2    T 4080  ☒ 4089 
Q 3630+71   T 3701  ☒ 3711 
Q 37+39     T 76    ☑ 76   
Q 2+382     T 384   ☒ 385  
Q 210+69    T 279   ☒ 288  
Q 3+794     T 797   ☑ 797  
Q 2564+24   T 2588  ☒ 2688 
Q 2830+92   T 2922  ☑ 2922 
Q 3868+62   T 3930  ☒ 3961 
Q 9059+737  T 9796  ☒ 9815 
Q 6+9521    T 9527  ☑ 9527 
Q 2408+922  T 3330  ☒ 3280 
Q 9395+2261 T 11656 ☒ 11666
Q 66+413    T 479   ☑ 479  
Q 9+74      T 83    ☒ 84   
Q 6638+867  T 7505  ☒ 7485 
Q 80+9822   T 9902  ☑ 9902 
Q 54+510    T 564   ☑ 564  
Q 67+9166   T 9233  ☒ 9223 
Q 4471+6220 T 10691 ☒ 10122
Q 3240+0    T 3240  ☒ 3239 
Q 3004+84   T 3088  ☒ 3087 
Q 1321+594  T 1915  